<a href="https://colab.research.google.com/github/rhysdavies21/library/blob/master/financial_formulae_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Financial Formulae**

In [0]:
# Black's formula

from scipy.stats import norm
import math

def black(f_0, strike, expiry, vol, rate, c_p):
    d_plus = (math.log(f_0 / strike) + 0.5 * vol * vol * expiry)/ (vol / math.sqrt(expiry))
    d_minus = (math.log(f_0 / strike) - 0.5 * vol * vol * expiry)/ (vol / math.sqrt(expiry))
    if c_p == 'c':
            option_value = (math.exp((-rate)*expiry))*(f_0 * norm.cdf(d_plus) - strike *norm.cdf(d_minus))
    elif c_p == 'p':
            option_value = (math.exp((-rate)*expiry))*(strike * norm.cdf(-d_minus) - f_0 *norm.cdf(-d_plus))
    else:
            print('Check input')
    return option_value

In [0]:
# Run black_price

# Request inputs
# f_0 = float(input('Input f_0 = '))
# strike = float(input('strike = '))
# expiry = float(input('expiry = '))
# black_vol = float(input('black_vol = '))
# rate = float(input('rate = '))
c_p = str(input('call (c) or put (p) = '))

f_0 = 0.07
strike = 0.08
expiry = 1
black_vol = 0.2
rate = 0.05

# Price option
if c_p in ['c', 'p']:
  black_price = black(f_0, strike, expiry, black_vol, rate, c_p)
  print('\n' "black_price = "+"{:.3%}".format(black_price))
else:
  print('Please check input')

call (c) or put (p) = c

black_price = 0.214%


In [0]:
# Bachelier 

from scipy.stats import norm
import math

def dPlusBachelier(F_0 , y, expiry , vol):
    d_plus = (F_0 - y)/ (vol * math.sqrt(expiry))
    return d_plus

def bachelier(F_0,y,expiry,vol,rfr,expiry_1,isCall):
    option_value = 0
    if expiry * vol == 0.0:
        if isCall:
            option_value = max(F_0 - y, 0.0)
        else:
            option_value = max(y - F_0 , 0.0)
    else:
        d1 = dPlusBachelier(F_0 = F_0 , y = y, expiry = expiry ,vol = vol)
        if isCall:
             option_value = (math.exp((-rfr)*expiry_1))*((F_0 - y)* norm.cdf(d1) + vol * math.sqrt(expiry) * norm.cdf(d1))
        else:
             option_value = (math.exp((-rfr)*expiry_1))*((y-F_0) * norm.cdf(-d1) + vol * math.sqrt(expiry) * norm.cdf(-d1))
    return option_value


In [0]:
bachelier_price = bachelier(0.07,0.08,1,0.20,0.069394,1.25,"isCall")
print("bachelier_price = "+"{:.3%}".format(bachelier_price))

bachelier_price = 8.363%


In [0]:
import math 

def calc_z(alpha, beta, v, f, k):
  calc_z = (v/alpha) * (f/k) ** ((1-beta)/2) * math.log(f/k)
  return calc_z

def calc_x(z, rho):
  calc_x = math.log(  ( math.sqrt(1-2*rho*z+z*z) + z - rho ) / (1-rho) )
  return calc_x

def sigma_atm(alpha, beta, rho, v, f, k, T, z, x):
  denom_one = 1+ ((1-beta)**2)*(1/24) * ((math.log(f/k))**2) + ((1-beta)**4)*(1/1920) * ((math.log(f/k))**4)
  
  denom_two = (1+ ((1-beta)**2)*(1/24) * (alpha**2)/((f*k)**(1-beta)) + (1/4)*rho*beta*v*alpha*((f*k)**((1-beta)/2)) + (2-3*rho*rho)*(1/24)*v*v ) * T
  
  sigma_atm = alpha * (1/ ((f*k)**((1-beta)/2))) * (1/denom_one * z * (1/x)) * (1/denom_two)
  return sigma_atm 

In [0]:
# Output calcuation

# Input
alpha_val = 0.2
beta_val = 1
f_val = 0.07
k_val = 0.08
v_val = 0.02

rho_val = -0.3

T_val = 1

# Intermediate calculation
z_val = calc_z(alpha_val, beta_val, v_val, f_val, k_val)
x_val = calc_x(z_val, rho_val)
sigma_atm_val = sigma_atm(alpha_val, beta_val, rho_val, v_val,
                          f_val, k_val, T_val, z_val, x_val)

# Show output
print('z = ', z_val)
print('x(z) = ', x_val)
print('sigma_atm = ', sigma_atm_val)

z =  -0.013353139262452247
x(z) =  -0.013379592495821192
sigma_atm =  0.1996587141951907
